In [1]:
import os
import requests
import pandas as pd
from io import BytesIO
import dlt
from dlt.destinations import filesystem
import duckdb

print("requests version: " + str(requests.__version__))
print("pandas version: " + str(pd.__version__))
print("dlt version: " + str(dlt.__version__))
print("duckdb version: " + str(duckdb.__version__))

requests version: 2.32.5
pandas version: 3.0.0
dlt version: 1.21.0
duckdb version: 1.4.4


In [2]:
# -------------------------------
# SCHEMAS
# -------------------------------

YELLOW_RENAME = {
    "VendorID": "vendor_id",
    "tpep_pickup_datetime": "pickup_datetime",
    "tpep_dropoff_datetime": "dropoff_datetime",
    "passenger_count": "passenger_count",
    "trip_distance": "trip_distance",
    "RatecodeID": "rate_code",
    "store_and_fwd_flag": "store_and_fwd_flag",
    "payment_type": "payment_type",
    "fare_amount": "fare_amount",
    "extra": "extra",
    "mta_tax": "mta_tax",
    "tip_amount": "tip_amount",
    "tolls_amount": "tolls_amount",
    "improvement_surcharge": "imp_surcharge",
    "airport_fee": "airport_fee",
    "total_amount": "total_amount",
    "PULocationID": "pickup_location_id",
    "DOLocationID": "dropoff_location_id"
}

GREEN_RENAME = {
    "VendorID": "vendor_id",
    "lpep_pickup_datetime": "pickup_datetime",
    "lpep_dropoff_datetime": "dropoff_datetime",
    "store_and_fwd_flag": "store_and_fwd_flag",
    "RatecodeID": "rate_code",
    "passenger_count": "passenger_count",
    "trip_distance": "trip_distance",
    "fare_amount": "fare_amount",
    "extra": "extra",
    "mta_tax": "mta_tax",
    "tip_amount": "tip_amount",
    "tolls_amount": "tolls_amount",
    "ehail_fee": "ehail_fee",
    "airport_fee": "airport_fee",
    "total_amount": "total_amount",
    "payment_type": "payment_type",
    "trip_type": "trip_type",
    "improvement_surcharge": "imp_surcharge",
    "PULocationID": "pickup_location_id",
    "DOLocationID": "dropoff_location_id"
}

# Full schema objects including dtypes, rename map, column order, datetime columns
YELLOW_SCHEMA = {
    "dtypes": {
        "vendor_id": "Int64",
        "pickup_datetime": "datetime64[ns]",
        "dropoff_datetime": "datetime64[ns]",
        "passenger_count": "Int64",
        "trip_distance": "float64",
        "rate_code": "Int64",
        "store_and_fwd_flag": "string",
        "payment_type": "Int64",
        "fare_amount": "float64",
        "extra": "float64",
        "mta_tax": "float64",
        "tip_amount": "float64",
        "tolls_amount": "float64",
        "imp_surcharge": "float64",
        "airport_fee": "float64",
        "total_amount": "float64",
        "pickup_location_id": "Int64",
        "dropoff_location_id": "Int64",
        "data_file_year": "Int64",
        "data_file_month": "Int64"
    },
    "rename_map": YELLOW_RENAME,
    "columns": [
        "vendor_id", "pickup_datetime", "dropoff_datetime", "passenger_count", "trip_distance",
        "rate_code", "store_and_fwd_flag", "payment_type", "fare_amount", "extra", "mta_tax",
        "tip_amount", "tolls_amount", "imp_surcharge", "airport_fee", "total_amount",
        "pickup_location_id", "dropoff_location_id", "data_file_year", "data_file_month"
    ],
    "datetime_cols": ["pickup_datetime", "dropoff_datetime"]
}

GREEN_SCHEMA = {
    "dtypes": {
        "vendor_id": "Int64",
        "pickup_datetime": "datetime64[ns]",
        "dropoff_datetime": "datetime64[ns]",
        "store_and_fwd_flag": "string",
        "rate_code": "Int64",
        "passenger_count": "Int64",
        "trip_distance": "float64",
        "fare_amount": "float64",
        "extra": "float64",
        "mta_tax": "float64",
        "tip_amount": "float64",
        "tolls_amount": "float64",
        "ehail_fee": "float64",
        "airport_fee": "float64",
        "total_amount": "float64",
        "payment_type": "Int64",
        "distance_between_service": "float64",
        "time_between_service": "float64",
        "trip_type": "Int64",
        "imp_surcharge": "float64",
        "pickup_location_id": "Int64",
        "dropoff_location_id": "Int64",
        "data_file_year": "Int64",
        "data_file_month": "Int64"
    },
    "rename_map": GREEN_RENAME,
    "columns": [
        "vendor_id", "pickup_datetime", "dropoff_datetime", "store_and_fwd_flag", "rate_code",
        "passenger_count", "trip_distance", "fare_amount", "extra", "mta_tax", "tip_amount",
        "tolls_amount", "ehail_fee", "airport_fee", "total_amount", "payment_type",
        "distance_between_service", "time_between_service", "trip_type", "imp_surcharge",
        "pickup_location_id", "dropoff_location_id", "data_file_year", "data_file_month"
    ],
    "datetime_cols": ["pickup_datetime", "dropoff_datetime"]
}

In [3]:
# -------------------------------
# HELPER FUNCTIONS
# -------------------------------

def generate_file_urls(data_type = "yellow", start_year = 2019, end_year = 2020):
    urls = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            month_str = f"{month:02d}"
            file_name = f"{data_type}_tripdata_{year}-{month_str}.csv.gz"
            url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/{data_type}/{file_name}"
            urls.append((file_name, url))
    return urls

def apply_schema(df: pd.DataFrame, schema: dict, year: int, month: int) -> pd.DataFrame:
    # Rename columns
    df = df.rename(columns = schema["rename_map"])

    # Add missing columns
    for col in schema["columns"]:
        if col not in df.columns:
            df[col] = pd.NA

    # Add year/month
    df["data_file_year"] = year
    df["data_file_month"] = month

    # Datetime conversion
    for col in schema["datetime_cols"]:
        df[col] = pd.to_datetime(df[col], errors = "coerce")

    # Numeric conversion
    for col, dtype in schema["dtypes"].items():
        if col in schema["datetime_cols"]:
            continue
        if "float" in dtype or "int" in dtype:
            df[col] = pd.to_numeric(df[col], errors = "coerce")

    # String conversion
    for col, dtype in schema["dtypes"].items():
        if dtype == "string":
            df[col] = df[col].astype("string")

    # Final dtype enforcement
    df = df.astype(schema["dtypes"])

    # Reorder columns
    df = df[schema["columns"]]

    return df

In [4]:
# -------------------------------
# DLT SOURCE
# -------------------------------

def make_taxi_source(data_type = "yellow"):
    schema = YELLOW_SCHEMA if data_type == "yellow" else GREEN_SCHEMA

    @dlt.source(name = f"{data_type}_tripdata")
    def taxi_source():
        dfs = []
        for file_name, url in generate_file_urls(data_type):
            print(f"Downloading {file_name}...")

            # Extract year/month
            year_month = file_name.split("_")[-1].replace(".csv.gz","")
            year, month = map(int, year_month.split("-"))

            # Retry loop
            for attempt in range(3):
                try:
                    response = requests.get(url)
                    if response.status_code != 200:
                        raise ValueError(f"File not found (status {response.status_code})")

                    df = pd.read_csv(
                        BytesIO(response.content),
                        compression = "gzip",
                        dtype = {"store_and_fwd_flag": "string"},
                        low_memory = False
                    )

                    # Apply schema
                    df = apply_schema(df, schema, year, month)

                    dfs.append(df)
                    break

                except Exception as e:
                    print(f"Attempt {attempt + 1} failed for {file_name}: {e}")
                    if attempt == 2:
                        print(f"Skipping {file_name} after 3 failed attempts.")
                    else:
                        print("Retrying...")
                            # Yield as DLT resource

        yield dlt.resource(dfs, name = f"{data_type}_tripdata")

    return taxi_source

In [ ]:
# -------------------------------
# PIPELINE
# -------------------------------

pipeline = dlt.pipeline(
    pipeline_name = "taxi_data",
    destination = "duckdb",
    dataset_name = "nytaxi"
)

# Example: create source and run
yellow_source = make_taxi_source("yellow")
green_source = make_taxi_source("green")

In [6]:
# Load yellow data
yellow_info = pipeline.run(yellow_source())

print("\nYellow taxi data load info:")
print(yellow_info)


Yellow taxi data load info:
Pipeline taxi_data load step completed in 11.74 seconds
1 load package(s) were loaded to destination duckdb and into dataset nytaxi
The duckdb destination used duckdb:////Users/rob/Projects/GitHub/data-engineering-zoomcamp/04-analytics-engineering/taxi_data.duckdb location to store data
Load package 1771020907.7037911 is LOADED and contains no failed jobs


In [7]:
# Load green data
green_info = pipeline.run(green_source())

print("Green taxi data load info:")
print(green_info)

Green taxi data load info:
Pipeline taxi_data load step completed in 1.85 seconds
2 load package(s) were loaded to destination duckdb and into dataset nytaxi
The duckdb destination used duckdb:////Users/rob/Projects/GitHub/data-engineering-zoomcamp/04-analytics-engineering/taxi_data.duckdb location to store data
Load package 1771021010.1064699 is LOADED and contains no failed jobs
Load package 1771021013.110284 is LOADED and contains no failed jobs


In [8]:
# Connect to the DuckDB file
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Optional: set search path to your dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# List all tables in the dataset
tables = conn.sql("SHOW TABLES").df()
print("Tables in DuckDB:")
print(tables)

# Check how many rows were loaded in yellow_tripdata
yellow_count = conn.sql("SELECT COUNT(*) AS row_count FROM yellow_tripdata").df()
print("Yellow tripdata row count:")
print(yellow_count)

# Check how many rows were loaded in green_tripdata
green_count = conn.sql("SELECT COUNT(*) AS row_count FROM green_tripdata").df()
print("Green tripdata row count:")
print(green_count)

Tables in DuckDB:
                  name
0           _dlt_loads
1  _dlt_pipeline_state
2         _dlt_version
3       green_tripdata
4      yellow_tripdata
Yellow tripdata row count:
   row_count
0  109047518
Green tripdata row count:
   row_count
0    7778101


In [9]:
# Preview 10 records from yellow
yellow_preview = conn.sql("SELECT * FROM yellow_tripdata LIMIT 10").df()
print("Yellow taxi sample:")
print(yellow_preview)

Yellow taxi sample:
   vendor_id           pickup_datetime          dropoff_datetime  \
0          1 2018-12-31 18:46:40-06:00 2018-12-31 18:53:20-06:00   
1          1 2018-12-31 18:59:47-06:00 2018-12-31 19:18:59-06:00   
2          2 2018-12-21 07:48:30-06:00 2018-12-21 07:52:40-06:00   
3          2 2018-11-28 09:52:25-06:00 2018-11-28 09:55:45-06:00   
4          2 2018-11-28 09:56:57-06:00 2018-11-28 09:58:33-06:00   
5          2 2018-11-28 10:25:49-06:00 2018-11-28 10:28:26-06:00   
6          2 2018-11-28 10:29:37-06:00 2018-11-28 10:33:43-06:00   
7          1 2018-12-31 18:21:28-06:00 2018-12-31 18:28:37-06:00   
8          1 2018-12-31 18:32:01-06:00 2018-12-31 18:45:39-06:00   
9          1 2018-12-31 18:57:32-06:00 2018-12-31 19:09:32-06:00   

   passenger_count  trip_distance  rate_code store_and_fwd_flag  payment_type  \
0                1            1.5          1                  N             1   
1                1            2.6          1                  N      

In [10]:
# Preview 10 records from green
green_preview = conn.sql("SELECT * FROM nytaxi.green_tripdata LIMIT 10").df()
print("\nGreen taxi sample:")
print(green_preview)


Green taxi sample:
   vendor_id           pickup_datetime          dropoff_datetime  \
0          2 2018-12-21 09:17:29-06:00 2018-12-21 09:18:57-06:00   
1          2 2018-12-31 18:10:16-06:00 2018-12-31 18:16:32-06:00   
2          2 2018-12-31 18:27:11-06:00 2018-12-31 18:31:38-06:00   
3          2 2018-12-31 18:46:20-06:00 2018-12-31 19:04:54-06:00   
4          2 2018-12-31 18:19:06-06:00 2018-12-31 18:39:43-06:00   
5          2 2018-12-31 18:12:35-06:00 2018-12-31 18:19:09-06:00   
6          2 2018-12-31 18:47:55-06:00 2018-12-31 19:00:01-06:00   
7          1 2018-12-31 18:12:47-06:00 2018-12-31 18:30:50-06:00   
8          2 2018-12-31 18:16:23-06:00 2018-12-31 18:39:46-06:00   
9          2 2018-12-31 18:58:02-06:00 2018-12-31 19:19:02-06:00   

  store_and_fwd_flag  rate_code  passenger_count  trip_distance  fare_amount  \
0                  N          1                5           0.00          3.0   
1                  N          1                2           0.86        

In [11]:
# Yellow taxi counts by year and month
yellow_counts = conn.sql("""
    SELECT data_file_year, data_file_month, COUNT(*) AS num_rows
    FROM yellow_tripdata
    GROUP BY data_file_year, data_file_month
    ORDER BY data_file_year, data_file_month
""").df()

print("Yellow taxi counts by year/month:")
print(yellow_counts)

Yellow taxi counts by year/month:
    data_file_year  data_file_month  num_rows
0             2019                1   7667792
1             2019                2   7019375
2             2019                3   7832545
3             2019                4   7433139
4             2019                5   7565261
5             2019                6   6941024
6             2019                7   6310419
7             2019                8   6073357
8             2019                9   6567788
9             2019               10   7213891
10            2019               11   6878111
11            2019               12   6896317
12            2020                1   6405008
13            2020                2   6299354
14            2020                3   3007292
15            2020                4    237993
16            2020                5    348371
17            2020                6    549760
18            2020                7    800412
19            2020                8   1007284


In [12]:
# Green taxi counts by year and month
green_counts = conn.sql("""
    SELECT data_file_year, data_file_month, COUNT(*) AS num_rows
    FROM green_tripdata
    GROUP BY data_file_year, data_file_month
    ORDER BY data_file_year, data_file_month
""").df()
print("\nGreen taxi counts by year/month:")
print(green_counts)


Green taxi counts by year/month:
    data_file_year  data_file_month  num_rows
0             2019                1    630918
1             2019                2    575685
2             2019                3    601102
3             2019                4    514392
4             2019                5    504887
5             2019                6    471052
6             2019                7    470743
7             2019                8    449695
8             2019                9    449063
9             2019               10    476386
10            2019               11    449500
11            2019               12    450627
12            2020                1    447770
13            2020                2    398632
14            2020                3    223406
15            2020                4     35612
16            2020                5     57360
17            2020                6     63109
18            2020                7     72257
19            2020                8     81063
